# House rocket analysis

In [ ]:
## 1.0 Imports
from IPython.core.display import HTML
from matplotlib           import pyplot       as plt
from matplotlib           import pyplot as plt
from matplotlib           import gridspec
from multiprocessing      import Pool
import defs  #Biblioteca feita para esse projeto. 
import pandas         as pd
import plotly.express as px
import seaborn        as sns
import time

## 2.0 Functions
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [20, 10]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    pd.set_option( 'display.float_format', lambda x: '%.2f' % x)
    sns.set()
    
def features ( data ):
    data['date']           = pd.to_datetime(data['date']).dt.strftime('%Y-%m-%d')
    data['dormitory_type'] = data['bedrooms'].apply( lambda x: 'studio' if x == 1 else 'apartament' if x == 2 else 'house' if x >= 3 else 'NA')
    data['condition_type'] = data['condition'].apply( lambda x: 'bad' if x <= 2 else 'regular' if (x == 3)|(x == 4) else 'good')
    data['yr_renovated']   = data['yr_renovated'].apply( lambda x: pd.to_datetime('1900-01-01',format = '%Y-%m-%d') if x == 0 else pd.to_datetime(x,format = '%Y'))
    data['nivel']          = data['price'].apply ( lambda x: 0 if x <= 321950 else 1 if ( x > 321950 )&( x <= 450000 ) else 2 if ( x > 450000 )&( x <= 645000 ) else 3)
    data                   = data.drop('sqft_living15', axis = 1)
    data                   = data.drop('sqft_lot15', axis = 1)
    
    return data

def show_dimensions( data ):
    print( f'Number of rows:{data.shape[0]}')
    print( f'Number of columns:{data.shape[1]}')
    
    return None
    
def collect_geodata( data ):
    # Geocode
    # Será utilizado programação paralela para melhor otimização do processo.
    data['query'] = data[['lat','long']].apply( lambda x: str( x['lat'] ) + ',' + str( x['long'] ), axis = 1 )
    
    #Number of cores;
    p = Pool ( 3 )
    
    start = time.process_time()
    data[['road', 'house_number', 'neighbourhood', 'city', 'county', 'state','place_id','osm_type','country','country_code']] = p.map( defs.get_data, data.iterrows() )
    end = time.process_time()
    print(f'Time elapsed {end - start}')
    
    return data
    
def plot_map ( data ):
    
    #Creating map
    houses = px.scatter_mapbox( data, 
                          lat = 'lat', 
                          lon = 'long', 
                          hover_name = 'road',
                          text = 'house_number',
                          size = 'price', 
                          color = 'nivel', 
                          opacity = 0.7,
                          color_continuous_scale = px.colors.cyclical.IceFire, 
                          zoom = 10, size_max = 15)
    houses.update_layout( mapbox_style = 'open-street-map')
    houses.update_layout( height = 600, margin = {'r':0,'t':0,'b':0,'l':0})
    houses.show()
    
    return None
    
def graphs_commercial( data ):
    # Commercial Attributes:
    fig = plt.figure( figsize = (25,12) )
    specs = gridspec.GridSpec( ncols = 2, nrows = 2, figure = fig)

    ax1 = fig.add_subplot( specs [0,:] ) #First line
    ax2 = fig.add_subplot( specs [1,0] ) #Second line, first column
    ax3 = fig.add_subplot( specs [1,1] ) #Second line, second column

    # price by year built
    df = data[['price','yr_built']].groupby('yr_built').mean().reset_index()
    sns.lineplot(x = 'yr_built', y = 'price', data = df, ax = ax1).set(title = 'Price by Year Built')
    ax1.set(xlabel = 'Year Built', ylabel = 'Price')

    # Price by bedrooms
    df = data[['bedrooms','price']].groupby('bedrooms').mean().reset_index()
    sns.barplot( x = 'bedrooms', y = 'price', data = df, ax = ax2).set(title = 'Price by bedrooms')
    ax2.set(xlabel = 'Number of bedrooms', ylabel = 'Price')

    # Price by bedrooms
    df = data[['yr_built', 'sqft_lot']].groupby('yr_built').mean().reset_index()
    sns.barplot( x = 'yr_built', y = 'sqft_lot', data = df, ax = ax3).set(title = 'Sqft by year built')
    ax3.set(xlabel = 'Year built', ylabel = 'Sqft_lot')
    plt.xticks(rotation = 90);
    
    return None

def graphs_attributes( data ):
    # Houses attributes

    fig = plt.figure( figsize = (25,12) )
    specs = gridspec.GridSpec( ncols = 2, nrows = 2, figure = fig)

    ax1 = fig.add_subplot( specs [0,:] ) #First line
    ax2 = fig.add_subplot( specs [1,0] ) #Second line, first column
    ax3 = fig.add_subplot( specs [1,1] ) #Second line, second column

    # Houses by bedrooms
    df = data[['bedrooms','id']].groupby('bedrooms').count().reset_index()
    sns.barplot( x = 'bedrooms', y = 'id', data = df, ax = ax1).set(title = 'Houses by bedrooms')
    ax1.set( xlabel = 'Number of bedrooms', ylabel = 'Number of houses')

    # Houses by floors
    df = data[['floors','id']].groupby('floors').count().reset_index()
    sns.barplot( x = 'floors', y = 'id', data = df, ax = ax2).set(title = 'Houses by floor')
    ax2.set( xlabel = 'Number of floors', ylabel = 'Number of houses')

    # Houses by bathrooms
    df = data[['bathrooms','id']].groupby('bathrooms').count().reset_index()
    sns.barplot( x = 'bathrooms', y = 'id', data = df, ax = ax3).set(title = 'Houses by bathrooms')
    ax3.set( xlabel = 'Number of bathrooms', ylabel = 'Number of houses')
    
    return None

## 3.0 ETL
jupyter_settings()
# Extract
data = pd.read_csv('datasets/kc_house_data.csv')
show_dimensions( data )
data.describe()

# Transform
features( data )
collect_geodata( data )

# Load
plot_map( data )
graphs_commercial( data )
graphs_attributes( data )
data.to_csv('datasets/kc_house_data.csv')